In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk import word_tokenize, sent_tokenize, pos_tag, pos_tag_sents
import gensim
import gensim.corpora as corpora
import gensim.models.ldamodel as ldamodel
from gensim.summarization import summarize
import pyLDAvis.gensim
import seaborn as sns
from wordcloud import WordCloud
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.probability import FreqDist
from IPython.utils.text import columnize
import warnings
warnings.filterwarnings("ignore")
import textwrap
%matplotlib notebook
plt.style.use('seaborn-darkgrid')



/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
#source of iso code database: https://unstats.un.org/unsd/methodology/m49/overview/
##The dataset represents country names as 3-letter ISO-alpha3 Codes.
# To convert these codes into country names, I merged df with UN country names
# This dataset also specifies the region (continent), which I will need later.

df = pd.read_csv('un-general-debates.csv').drop('session', axis=1)
country_names = pd.read_excel('UNSD — Methodology.xlsx')
df = pd.merge(df, country_names[['Region Name','Country or Area','ISO-alpha3 Code']],
             how='left', left_on='country', right_on='ISO-alpha3 Code')
df.drop('ISO-alpha3 Code',axis=1, inplace=True) #removing a duplicate column
df.head()

,year,country,text,Region Name,Country or Area
0,1989,MDV,﻿It is indeed a pleasure for me and the member...,Asia,Maldives
1,1989,FIN,"﻿\nMay I begin by congratulating you. Sir, on ...",Europe,Finland
2,1989,NER,"﻿\nMr. President, it is a particular pleasure ...",Africa,Niger
3,1989,URY,﻿\nDuring the debate at the fortieth session o...,Americas,Uruguay
4,1989,ZWE,﻿I should like at the outset to express my del...,Africa,Zimbabwe


In [ ]:
# Preparing texts for analysis:
def preprocessText(text):
    text = re.sub('[^a-z]+',' ', text.lower()) # remove all non-letter characters
    tokens = word_tokenize(text) # returns a list of individual words
    # Removing unhelpfull, ubiquitous words ('stop words', e.g. ‘the’, ‘is’, ‘are’):
    tokens = [token for token in tokens if len(token) > 4 and
             token not in nltk.corpus.stopwords.words('english')]
    # Lemmatizing removes inflectional endings and gets the root word (lemma):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

# processed text as list of lemmas:
df['lemmatized_tokens'] = df['text'].apply(preprocessText)
# processed text in continous form:
df['lemmatized_text'] = [' '.join(x) for x in df['lemmatized_tokens']]
# calculating the number of times each words appears in a speech:
df['freq_dist'] = df['lemmatized_tokens'].apply(FreqDist)
df[['text','lemmatized_tokens','lemmatized_text']].head()